In [1]:
#!pip install pymysql
#!pip install mysql.connector
import pandas as pd
import pymysql
import mysql.connector
from sqlalchemy import create_engine, text

# Connect to MySQL database
username = 'root'  
password = '12345678' 
database = 'linkedin'  
host = 'localhost'  
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')
mycursor = engine.connect()

try:
    mycursor.execute(text("SET FOREIGN_KEY_CHECKS=0;"))  
    mycursor.execute(text("DROP TABLE IF EXISTS job_skill;"))
    mycursor.execute(text("DROP TABLE IF EXISTS job;"))
    mycursor.execute(text("DROP TABLE IF EXISTS skills;"))
    mycursor.execute(text("DROP TABLE IF EXISTS address;"))
    mycursor.execute(text("SET FOREIGN_KEY_CHECKS=1;")) 

    # Create tables
    mycursor.execute(text("""
    CREATE TABLE address (
        address_id INT AUTO_INCREMENT PRIMARY KEY,
        job_location VARCHAR(100),
        city VARCHAR(50),
        country VARCHAR(100)
    );
    """))

    mycursor.execute(text("""
      CREATE TABLE job (
        job_link VARCHAR(600) PRIMARY KEY,
        address_id INT,
        title VARCHAR(255),
        company VARCHAR(255),
        position VARCHAR(255),
        job_level VARCHAR(50),
        job_type VARCHAR(50),
        FOREIGN KEY (address_id) REFERENCES address(address_id)
    );

    """))

    mycursor.execute(text("""
    CREATE TABLE skills (
        skill_id INT AUTO_INCREMENT PRIMARY KEY,
        skill TEXT
    );
    """))

    mycursor.execute(text("""
    CREATE TABLE job_skill (
        job_skill_id int,
        job_link VARCHAR(600),
        skill_id INT,
        PRIMARY KEY (job_skill_id),
        FOREIGN KEY (job_link) REFERENCES job(job_link),
        FOREIGN KEY (skill_id) REFERENCES skills(skill_id)
    );
    """))

except Exception as e:
    print("An error occurred:", e)
finally:
    # Close connection
    mycursor.close()


In [2]:
summary = pd.read_csv("job_summary.csv")
skill = pd.read_csv("job_skills.csv")
posting = pd.read_csv("linkedin_job_postings.csv")

In [3]:
posting = posting[['job_link', 'job_title', 'company', 'job_location', 'search_city', 'search_country', 'search_position', 'job_level', 'job_type']]
posting = posting.rename(columns = {'job_title': 'title', 'search_city': 'city', 'search_country': 'country', 'search_position': 'position'})

In [4]:
posting.head(3)

,job_link,title,company,job_location,city,country,position,job_level,job_type
0,https://www.linkedin.com/jobs/view/account-exe...,Account Executive - Dispensing (NorCal/Norther...,BD,"San Diego, CA",Coronado,United States,Color Maker,Mid senior,Onsite
1,https://www.linkedin.com/jobs/view/registered-...,Registered Nurse - RN Care Manager,Trinity Health MI,"Norton Shores, MI",Grand Haven,United States,Director Nursing Service,Mid senior,Onsite
2,https://www.linkedin.com/jobs/view/restaurant-...,RESTAURANT SUPERVISOR - THE FORKLIFT,Wasatch Adaptive Sports,"Sandy, UT",Tooele,United States,Stand-In,Mid senior,Onsite


In [5]:
posting.title.value_counts()

title
LEAD SALES ASSOCIATE-FT                                  2714
Customer Service Representative                          2134
Assistant Manager                                        2009
Store Manager                                            1745
LEAD SALES ASSOCIATE-PT                                  1721
                                                         ... 
Senior Ecological Consultant                                1
Pool Supervisor - Seasonal                                  1
Lead Assistant Supervisor (Blow Molding)                    1
Manager Promotion Operations                                1
Personal Injury Trial Attorney (10+ years experience)       1
Name: count, Length: 226476, dtype: int64

In [6]:
posting.country.value_counts()

country
United States     371415
United Kingdom     37083
Canada             18206
Australia           9882
Name: count, dtype: int64

In [7]:
posting.position.value_counts()

position
Christian Science Nurse                             7164
Circulation-Sales Representative                    7144
Extra                                               6212
Change Person                                       5882
Consultant Education                                5190
                                                    ... 
Supervisor Swimming-Pool Maintenance                   1
Sales Representative Material-Handling Equipment       1
Booster Assembler                                      1
Cabinet Assembler                                      1
Librarian Special Collections                          1
Name: count, Length: 1841, dtype: int64

In [8]:
posting.job_type.value_counts()

job_type
Onsite    433802
Hybrid      1664
Remote      1120
Name: count, dtype: int64

In [9]:
common_job_link = set(summary['job_link']).intersection(set(skill['job_link']), set(posting['job_link']))
print("common_job_link type:", type(common_job_link))
print("Total number of common job_link:", len(common_job_link))
print("\n")

common_job_link type: <class 'set'>
Total number of common job_link: 6080




In [10]:
print("Summary dataset original dimension:", summary.shape)
summary = summary[summary['job_link'].isin(common_job_link)]
print("Filtered Summary dataset dimension:", summary.shape)

Summary dataset original dimension: (59999, 2)
Filtered Summary dataset dimension: (6080, 2)


In [11]:
print("Skill dataset original dimension:", skill.shape)
skill = skill[skill['job_link'].isin(common_job_link)]
print("Filtered Skill dataset dimension:", skill.shape)

Skill dataset original dimension: (679699, 2)
Filtered Skill dataset dimension: (6080, 2)


In [12]:
print("Posting dataset original dimension:", posting.shape)
posting = posting[posting['job_link'].isin(common_job_link)]
print("Filtered Posting dataset dimension:", posting.shape)

Posting dataset original dimension: (436586, 9)
Filtered Posting dataset dimension: (6080, 9)


In [13]:
linkedin = pd.merge(summary, skill, on='job_link', how='inner')
linkedin = pd.merge(linkedin, posting, on='job_link', how='inner')
linkedin = linkedin.rename(columns = {'job_summary': 'summary', 'job_skills': 'skill'})
linkedin.head(5)

,job_link,summary,skill,title,company,job_location,city,country,position,job_level,job_type
0,https://www.linkedin.com/jobs/view/material-ha...,Job Details:\nJob Title : Quality Liaison (599...,"Inspection, Rework, Reporting, Communication, ...",Material Handler,Intelliswift Software,"Austin, TX",Austin,United States,Handler,Mid senior,Onsite
1,https://www.linkedin.com/jobs/view/middle-scho...,Position:\nMiddle School Soccer Coach\nLocatio...,"Soccer Coaching, Coaching Tactics, Practice Dr...",Middle School Soccer Coach [Sexton],Lansing School District,"Lansing, MI",East Lansing,United States,Head Coach,Mid senior,Onsite
2,https://www.linkedin.com/jobs/view/boys-basket...,SEASONAL ONLY POSITION\nIn order to be conside...,"Athletic coaching certificate, Teaching certif...",Boys Basketball Coach,Leon County Schools,"Tallahassee, FL",Tallahassee,United States,Head Coach,Mid senior,Onsite
3,https://www.linkedin.com/jobs/view/store-manag...,Our Store Managers lead teams to maximize the ...,"Team Leadership, Optimization, Financial Outpu...",Store Manager,Harbor Freight Tools,"Moundsville, WV",West Virginia,United States,Head Coach,Mid senior,Onsite
4,https://www.linkedin.com/jobs/view/store-manag...,Dollar General Corporation has been delivering...,"Inventory management, Merchandising, Cashier a...",STORE MANAGER CANDIDATE,Dollar General,"Houston, TX",Baytown,United States,Head Coach,Mid senior,Onsite


In [14]:
job_skill = linkedin[['job_link']]

In [15]:
skill_column = linkedin[['skill']]

In [16]:
skill_column = skill_column.skill.str.split(',').apply(pd.Series, 1).stack()
# Drop index level to match main dataframe
skill_column.index = skill_column.index.droplevel(-1)
skill_column

0               Inspection
0                   Rework
0                Reporting
0            Communication
0              Negotiation
               ...        
6079                Profit
6079      Customer Service
6079       Team Leadership
6079         Communication
6079     People Management
Length: 117185, dtype: object

In [17]:
unique_skill = pd.DataFrame(skill_column.str.lower().str.strip()).rename(columns = {0:'skill'}).drop_duplicates()
unique_skill.insert(0, 'skill_id', range(1, 1 + len(unique_skill)))
unique_skill

,skill_id,skill
0,1,inspection
0,2,rework
0,3,reporting
0,4,communication
0,5,negotiation
...,...,...
6078,41036,multifamily development
6078,41037,project oversight
6078,41038,daytoday operations supervision
6078,41039,task completion accountability


In [18]:
skill_df = pd.DataFrame(skill_column.str.lower().str.strip())
skill_df = skill_df.rename_axis('index').reset_index()
skill_df

,index,0
0,0,inspection
1,0,rework
2,0,reporting
3,0,communication
4,0,negotiation
...,...,...
117180,6079,profit
117181,6079,customer service
117182,6079,team leadership
117183,6079,communication


In [19]:
job_link_df = linkedin.job_link
job_link_df = job_link_df.rename_axis('index').reset_index()
job_link_df

,index,job_link
0,0,https://www.linkedin.com/jobs/view/material-ha...
1,1,https://www.linkedin.com/jobs/view/middle-scho...
2,2,https://www.linkedin.com/jobs/view/boys-basket...
3,3,https://www.linkedin.com/jobs/view/store-manag...
4,4,https://www.linkedin.com/jobs/view/store-manag...
...,...,...
6075,6075,https://www.linkedin.com/jobs/view/retail-supe...
6076,6076,https://www.linkedin.com/jobs/view/full-time-r...
6077,6077,https://www.linkedin.com/jobs/view/multifamily...
6078,6078,https://www.linkedin.com/jobs/view/superintend...


In [20]:
job_skill = pd.merge(skill_df, job_link_df, on='index', how='inner')[["job_link", 0]]
job_skill = job_skill.rename(columns={0: "skill"})
job_skill.skill = job_skill.skill.str.lower().str.strip()
job_skill = pd.merge(unique_skill, job_skill, on='skill', how='inner')
job_skill.insert(0, 'job_skill_id', range(1, 1 + len(job_skill)))

job_skill_df = pd.DataFrame(skill_column.str.lower().str.strip()).rename(columns={0: 'skill'}).drop_duplicates()
job_skill_df.insert(0, 'skill_id', range(1, 1 + len(job_skill_df)))

job_skill = job_skill.drop_duplicates()
job_skill = job_skill.drop(columns=['skill'])

job_skill.head(5)

,job_skill_id,skill_id,job_link
0,1,1,https://www.linkedin.com/jobs/view/material-ha...
1,2,1,https://www.linkedin.com/jobs/view/line-leader...
2,3,1,https://www.linkedin.com/jobs/view/lead-recrea...
3,4,1,https://www.linkedin.com/jobs/view/production-...
4,5,1,https://www.linkedin.com/jobs/view/field-inspe...


In [21]:
address_df = linkedin[['job_location', 'city', 'country']].drop_duplicates()
address_df.insert(0, 'address_id', range(1, 1 + len(address_df)))
address_df.head(5)

,address_id,job_location,city,country
0,1,"Austin, TX",Austin,United States
1,2,"Lansing, MI",East Lansing,United States
2,3,"Tallahassee, FL",Tallahassee,United States
3,4,"Moundsville, WV",West Virginia,United States
4,5,"Houston, TX",Baytown,United States


In [22]:
# Joining address_id to the whole dataframe
linkedin = pd.merge(linkedin, address_df, on=['job_location', 'city', 'country'], how='inner')
linkedin.head(1)

,job_link,summary,skill,title,company,job_location,city,country,position,job_level,job_type,address_id
0,https://www.linkedin.com/jobs/view/material-ha...,Job Details:\nJob Title : Quality Liaison (599...,"Inspection, Rework, Reporting, Communication, ...",Material Handler,Intelliswift Software,"Austin, TX",Austin,United States,Handler,Mid senior,Onsite,1


In [23]:
address_df.to_sql(name='address', con=engine, if_exists='append', index=False)
print( "Address Table Finished!")

Address Table Finished!


In [24]:
print(linkedin.columns)
linkedin['job_link'].str.len().max()

Index(['job_link', 'summary', 'skill', 'title', 'company', 'job_location',
       'city', 'country', 'position', 'job_level', 'job_type', 'address_id'],
      dtype='object')


244

In [25]:
linkedin[['job_link', 'address_id', 'title', 'company', 'position', 'job_level', 'job_type']].to_sql(name='job', con=engine, if_exists='append', index=False)
print("Job Table Finished!")

Job Table Finished!


In [26]:
unique_skill['skill'].str.len().max()
unique_skill.to_sql(name = 'skills', con = engine, if_exists = 'append', index = False)
print("Skills Table Finished!")

Skills Table Finished!


In [27]:
job_skill.to_sql(name = 'job_skill', con = engine, if_exists = 'append', index = False)
print("Job_skill table Finished!")

Job_skill table Finished!
